In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
!nvidia-smi

Sun Aug 10 08:45:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
# Install core libraries for the RAG prototype.
# -qU = quiet output + upgrade to latest compatible versions
# langchain            : framework for chaining LLM + retriever steps
# langchain-community  : community loaders/vector stores (e.g., DirectoryLoader, FAISS)
# langchain-openai     : OpenAI wrappers (ChatOpenAI, OpenAIEmbeddings)
# tiktoken             : OpenAI tokenizer for token counting/chunking
!pip install -qU langchain langchain-community langchain-openai tiktoken

In [20]:
# Install FAISS with GPU
# Use only if your Colab runtime has an NVIDIA GPU (check using command "!nvidia-smi").
# -qU = quiet + upgrade. If this fails (CUDA mismatch/no GPU), use "pip install -qU faiss-cpu"
!pip install -qU faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [21]:
# Using CPU, as GPU command doesn't work.
!pip install -qU faiss-cpu

In [22]:
# Quick environment sanity-check: print Python version
import sys, subprocess, importlib, os
print("Python:", sys.version.split()[0])

# Helper: print the installed version of a package (skip gracefully if unavailable)
def ver(pkg):
    try:
        from importlib.metadata import version  # standard way to fetch package version
        print(pkg, version(pkg))
    except Exception as e:
        print(pkg, "— version check skipped:", e)

# Report versions of core libraries used in this RAG prototype
ver("langchain"); ver("langchain-community"); ver("langchain-openai"); ver("tiktoken")

# Verify FAISS availability and show its version; catch import issues cleanly
try:
    import faiss
    print("faiss:", faiss.__version__ if hasattr(faiss, "__version__") else "ok")
except Exception as e:
    print("faiss import error:", e)

Python: 3.11.13
langchain 0.3.27
langchain-community 0.3.27
langchain-openai 0.3.29
tiktoken 0.11.0
faiss: 1.11.0


In [23]:
# Verify FAISS is installed and importable.
# This confirms a CPU-only install.
import faiss
print("FAISS OK (CPU)")

FAISS OK (CPU)


In [24]:
# Securely prompt for your OpenAI API key
import os
from getpass import getpass

# Save the key to an environment variable for this session
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# Confirm setup (does not print the key)
print("OpenAI key set")

Enter your OpenAI API key: ··········
OpenAI key set


In [25]:
# Verify files exist in Drive
# Lists all .txt files in the target folder so you can confirm names/count.
from pathlib import Path

DATA_DIR = Path("/content/drive/MyDrive/enterprise-rag-langchain-faiss/data")
files = sorted(DATA_DIR.glob("*.txt"))
print(f"Found {len(files)} .txt files:")
for p in files:
    print("•", p.name)

Found 5 .txt files:
• basel3_overview.txt.txt
• fatf_recs_overview.txt.txt
• iso20022_about.txt.txt
• open_banking_what_is.txt.txt
• psd2_intro.txt.txt


In [26]:
# Load the .txt files as LangChain Documents
# DirectoryLoader + TextLoader read files and attach basic metadata.
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    str(DATA_DIR),
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True,
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

100%|██████████| 5/5 [00:00<00:00, 506.36it/s]

Loaded 5 documents


In [27]:
# Preview of loaded content
# Prints each filename, character count, and a snippet for sanity-checks.
from pathlib import Path

for d in docs:
    src = Path(d.metadata.get("source", "?")).name
    snippet = d.page_content[:400].replace("\n", " ")
    print(f"\n— {src} ({len(d.page_content)} chars)")
    print(snippet + ("..." if len(d.page_content) > 400 else ""))


— basel3_overview.txt.txt (4888 chars)
The Basel III reforms have now been integrated into the consolidated Basel Framework, which comprises all of the current and forthcoming standards of the Basel Committee on Banking Supervision. For background, set out below are the main publications that describe the changes to the Basel Framework that were agreed as part of Basel III.  Basel III is an internationally agreed set of measures develo...

— psd2_intro.txt.txt (7129 chars)
The revised Payment Services Directive (PSD2) updates and enhances the EU rules put in place by the initial PSD adopted in 2007. The PSD2 entered into force on 12 January 2016 and EU Member States were given until 13 January 2018 to transpose it into national law.  The main objectives of the PSD2 are (i) to contribute to a more integrated and efficient European payments market; (ii) to further lev...

— fatf_recs_overview.txt.txt (55389 chars)
 INTERNATIONAL STANDARDS ON COMBATING MONEY LAUNDERING AND THE FINANCING 

In [28]:
# Choose chunk size and overlap (safe defaults for GPT context windows)
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)
print("Chunking params set.")


Chunking params set.


In [29]:
# Split the loaded documents (variable `docs`)
chunks = splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks from {len(docs)} documents.")

Created 110 chunks from 5 documents.


In [30]:
# Add a running chunk_id so answers can reference sources cleanly
for i, c in enumerate(chunks):
    c.metadata["chunk_id"] = i

# Quick peek
for c in chunks[:3]:
    src = c.metadata.get("source", "?")
    print(f"[chunk_id={c.metadata['chunk_id']}] {src} :: {len(c.page_content)} chars")


[chunk_id=0] /content/drive/MyDrive/enterprise-rag-langchain-faiss/data/basel3_overview.txt.txt :: 849 chars
[chunk_id=1] /content/drive/MyDrive/enterprise-rag-langchain-faiss/data/basel3_overview.txt.txt :: 804 chars
[chunk_id=2] /content/drive/MyDrive/enterprise-rag-langchain-faiss/data/basel3_overview.txt.txt :: 933 chars


In [31]:
# Check token sizes so we know they fit comfortably in prompts
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

lengths = [len(enc.encode(c.page_content)) for c in chunks]
print(
    f"Chunks: {len(chunks)} | tokens per chunk -> "
    f"min={min(lengths)}, p50={sorted(lengths)[len(lengths)//2]}, max={max(lengths)}"
)

Chunks: 110 | tokens per chunk -> min=9, p50=131, max=208


In [37]:
# SentenceTransformers is used to get local embeddings
!pip install -qU langchain-huggingface sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 11.2 MB/s eta 0:00:00


In [39]:
# Initialise Hugging Face embeddings (latest supported import)
from langchain_huggingface import HuggingFaceEmbeddings
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("Hugging Face embeddings ready (dim=384)")

Hugging Face embeddings ready (dim=384)


In [40]:
# Extract plain text content and metadata from each chunk for embedding
texts = [c.page_content for c in chunks]
metadatas = [c.metadata for c in chunks]
print(f"Prepared {len(texts)} chunks for embedding")

Prepared 110 chunks for embedding


In [41]:
# Generate local embeddings for each chunk and confirm vector dimensions
vectors = emb.embed_documents(texts)
dim = len(vectors[0]) if vectors else 0
print(f"Created {len(vectors)} embeddings, dim={dim}")

Created 110 embeddings, dim=384
